# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
import numpy as np
import pandas as pd
from keras.datasets import cifar10

#Load the dataset:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

C:\Users\Balajisri\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Balajisri\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Balajisri\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Balajisri\Anaconda3\lib\site-packages\tensorflow\p

In [2]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [3]:
X_set1 = []
y_set1 = []
X_set2 = []
y_set2 = []

for (a, b) in zip(X, y): 
    if (b>=0) and (b<=4):
        X_set1.append(a)
        y_set1.append(b)
    else:
        X_set2.append(a)
        y_set2.append(b)
        
X_set1 = np.asarray(X_set1)
y_set1 = np.asarray(y_set1)
X_set2 = np.asarray(X_set2)
y_set2 = np.asarray(y_set2)

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [4]:
len(np.unique(y_set1))

5

In [5]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_set1, y_set1, test_size=0.20, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_set2, y_set2, test_size=0.20, random_state=42)

In [6]:
num_classes = len(np.unique(y_train))
from keras.utils import np_utils
y_train1 = np_utils.to_categorical(y_train1, num_classes)
y_test1 = np_utils.to_categorical(y_test1, num_classes)
y_train2 = np_utils.to_categorical(y_train2, num_classes)
y_test2 = np_utils.to_categorical(y_test2, num_classes)

In [7]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [8]:
X_train1 = X_train1.astype('float32')
X_test1 = X_test1.astype('float32')
X_train1 /= 255
X_test1 /= 255

X_train2 = X_train2.astype('float32')
X_test2 = X_test2.astype('float32')
X_train2 /= 255
X_test2 /= 255

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [10]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu',
                 input_shape=X_train.shape[1:]))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3) , activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,name='output', activation='softmax'))
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
__________

In [12]:
batch_size = 32
model.fit(X_train1, y_train1,
              batch_size=batch_size,
              epochs=25,
              validation_data=(X_test1, y_test1),
              shuffle=True)

Train on 24000 samples, validate on 6000 samples
Epoch 1/25
24000/24000 [==============================] - 9s 371us/step - loss: 1.3271 - acc: 0.4415 - val_loss: 1.1454 - val_acc: 0.5370
Epoch 2/25
24000/24000 [==============================] - 6s 258us/step - loss: 1.0701 - acc: 0.5655 - val_loss: 0.9715 - val_acc: 0.6223
Epoch 3/25
24000/24000 [==============================] - 6s 255us/step - loss: 0.9638 - acc: 0.6086 - val_loss: 0.8980 - val_acc: 0.6422
Epoch 4/25
24000/24000 [==============================] - 6s 257us/step - loss: 0.9010 - acc: 0.6402 - val_loss: 0.8373 - val_acc: 0.6728
Epoch 5/25
24000/24000 [==============================] - 6s 259us/step - loss: 0.8507 - acc: 0.6653 - val_loss: 0.8076 - val_acc: 0.6918
Epoch 6/25
24000/24000 [==============================] - 6s 258us/step - loss: 0.8161 - acc: 0.6818 - val_loss: 0.7646 - val_acc: 0.7077
Epoch 7/25
24000/24000 [==============================] - 6s 258us/step - loss: 0.7858 - acc: 0.6975 - val_loss: 0.7865 - v

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [13]:
model2 = Sequential()

for layer in model.layers:
    model2.add(layer)

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model2.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
    
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
__________

In [14]:
for layer in model2.layers:
    if layer.name != 'output':
        layer.trainable = False

from termcolor import colored    #  python module for ANSII Color formatting for output in terminal

#Check which layers have been frozen 
for layer in model2.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

conv2d_1
False
conv2d_2
False
max_pooling2d_1
False
dropout_1
False
conv2d_3
False
conv2d_4
False
max_pooling2d_2
False
dropout_2
False
flatten_1
False
dense_1
False
dropout_3
False
output
True


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [15]:
model2.fit(X_train2, y_train2,
              batch_size=batch_size,
              epochs=15,
              validation_data=(X_test2, y_test2),
              shuffle=True)

C:\Users\Balajisri\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Train on 24000 samples, validate on 6000 samples
Epoch 1/15
24000/24000 [==============================] - 7s 279us/step - loss: 1.7244 - acc: 0.3165 - val_loss: 0.9920 - val_acc: 0.6288
Epoch 2/15
24000/24000 [==============================] - 6s 262us/step - loss: 0.9601 - acc: 0.6229 - val_loss: 0.7125 - val_acc: 0.7353
Epoch 3/15
24000/24000 [==============================] - 6s 258us/step - loss: 0.7533 - acc: 0.7157 - val_loss: 0.5610 - val_acc: 0.8012
Epoch 4/15
24000/24000 [==============================] - 6s 256us/step - loss: 0.6423 - acc: 0.7630 - val_loss: 0.4976 - val_acc: 0.8228
Epoch 5/15
24000/24000 [==============================] - 6s 256us/step - loss: 0.5656 - acc: 0.7923 - val_loss: 0.4767 - val_acc: 0.8312
Epoch 6/15
24000/24000 [==============================] - 6s 257us/step - loss: 0.5167 - acc: 0.8135 - val_loss: 0.4025 - val_acc: 0.8545
Epoch 7/15
24000/24000 [==============================] - 6s 257us/step - loss: 0.4730 - acc: 0.8287 - val_loss: 0.3824 - v

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [16]:
from sklearn.datasets import fetch_20newsgroups

In [17]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [18]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [19]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [20]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2], dtype=int64)

In [21]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [22]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [23]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

In [24]:
X_train, X_test, y_train, y_test = train_test_split(twenty_train.data, twenty_train.target, test_size=0.20, random_state=42)

In [25]:
len(y_test)

452

In [26]:
vect = TfidfVectorizer(stop_words='english')
X_train_tfidf = vect.fit_transform(X_train)
X_test_tfidf = vect.transform(X_test)

In [27]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [28]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train_tfidf, y_train)

C:\Users\Balajisri\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Balajisri\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [29]:
from sklearn import metrics
pred = logreg.predict(X_test_tfidf)
print(metrics.accuracy_score(y_test,pred))

0.9690265486725663
